In [1]:
### Load preprocessed data

In [2]:
# conda  uninstall tensorboard; pip uninstall -y tensorboard; conda install tensorboard; conda install pytorch-lightning -c conda-forge

In [3]:
#!python ../src/download_ml20.py

In [4]:
import numpy as np
fh = np.load('data/dataset_ml20_wide.npz')
# We have a bunch of feature columns and last column is the y-target
max_seq_len = 768 + 1
train_items = fh['train_items'].astype(np.int64)[:, :max_seq_len]
train_ratng = fh['train_ratng'].astype(np.int64)[:, :max_seq_len]
test_items = fh['test_items'].astype(np.int64)[:, :max_seq_len]
test_ratng = fh['test_ratng'].astype(np.int64)[:, :max_seq_len]

n_user = train_items.shape[0]
n_rank = train_items.shape[1]
n_item = int(train_items.max() + 1)
n_resp = int(train_ratng.max() + 1)

In [5]:
np.unique(train_ratng[train_items > 0])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [6]:
np.unique(train_items)

array([     0,      2,      3, ..., 131170, 131237, 131262])

In [7]:
train_items.shape, test_items.shape

((124644, 769), (13849, 769))

In [8]:
!pip install -q reformer_pytorch

In [9]:
import random
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch import from_numpy
import pytorch_lightning as pl
from torch.nn import functional as F
from reformer_pytorch import Reformer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [10]:
from abstract_model import AbstractModel

In [15]:
class AR(AbstractModel):
    def __init__(self, n_item, n_dim, n_resp, n_rank, 
                 train_x, train_y, test_x, test_y,
                 heads=2, depth=2, batch_size=32):
        super().__init__()
        self.n_dim = n_dim
        self.n_item = n_item
        self.n_resp = n_resp
        # This means that item=0 will always yield the zero vector
        self.item = nn.Embedding(n_item, n_dim, padding_idx=0)
        self.resp = nn.Embedding(n_resp, n_dim)
        self.reformer = Reformer(dim=n_dim, depth=depth, heads=heads, causal=True, max_seq_len=n_rank)
        self.user_lin = nn.Linear(n_dim, n_dim * n_resp)
        self.item_lin = nn.Linear(n_dim, n_dim * n_resp)
        self.save_data(train_x, train_y, test_x, test_y)
        self.batch_size = batch_size
    
    def forward(self, items, ratng):
        item_vec = self.item(items)
        resp_vec = self.resp(ratng)
        intx_vec = item_vec * resp_vec
        mask = items != 0
        user_vec = self.reformer(intx_vec, input_mask=mask)
        return user_vec
    
    def likelihood(self, user_vec, items, ratg):
        batchsize, window, n_dim = user_vec.shape
        item_vec = self.item(items)
        # Linearly map one user & item vector to one user &  item vector
        # per possible response type
        user_raw_resp = self.user_lin(user_vec).reshape((batchsize, window, n_dim, self.n_resp))
        item_raw_resp = self.item_lin(item_vec).reshape((batchsize, window, n_dim, self.n_resp))
        # remove first user element and call it the  bias, the  rest is the user vector
        # same for items
        user_bas_resp, user_vec_resp = user_raw_resp[:, :, 0, :], user_raw_resp[:, :, 1:, :]
        item_bas_resp, item_vec_resp = item_raw_resp[:, :, 0, :], item_raw_resp[:, :, 1:, :]
        # Sum interactions  across n_dim back down to  (batchsize, window, n_resp)
        scor = user_bas_resp + item_bas_resp + (user_vec_resp * item_vec_resp).sum(dim=2)
        scor_flat = scor.reshape((batchsize * window, self.n_resp))
        ratg_flat = ratg.reshape((batchsize * window))
        loss = F.cross_entropy(scor_flat, ratg_flat, ignore_index=0, reduction='mean')
        return loss
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, weight_decay=1e-5)
    
    def training_step(self, batch, batch_nb):
        (items,), ratng = batch
        # Pass in leading arrays, missing the last element
        # for every user that's to  be predicted
        user_vec = self.forward(items[:,  :-1], ratng[:, :-1])
        # Given previous tokens, predict the next interaction
        loss = self.likelihood(user_vec, items[:, 1:],  ratng[:, 1:])
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        dat = self.training_step(batch, batch_nb)
        dat['test_loss'] = dat.pop('loss')
        return dat

    
n_dim = 32
model = AR(n_item, n_dim, n_resp, n_rank, train_items, train_ratng, test_items, test_ratng,
           heads=8, depth=6, batch_size=48)

# add a logger
logger = TensorBoardLogger("tb_logs", name="ar_model")

trainer = pl.Trainer(max_epochs=5, gpus=1,
                     reload_dataloaders_every_epoch=True,
                     logger=logger)    

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [16]:
test_items.shape

(13849, 769)

### Train model

In [17]:
# trainer.test(model)

In [ ]:
trainer.fit(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [19]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.5363, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.5363019704818726}

In [20]:
trainer.fit(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


1

In [21]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.5233, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.5233385562896729}

In [22]:
trainer.fit(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


1

In [23]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.5117, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.511722207069397}

In [24]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.5042, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.5042461156845093}

In [25]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4994, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.4994462728500366}

In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [36]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4646, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.4646044969558716}

In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [39]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4620, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.4619640111923218}

In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [44]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4563, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.4563312530517578}

In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [62]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4421, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.442121982574463}

In [49]:
trainer.fit(model)
trainer.test(model)


  | Name     | Type      | Params
---------------------------------------
0 | item     | Embedding | 4 M   
1 | resp     | Embedding | 352   
2 | reformer | Reformer  | 69 K  
3 | user_lin | Linear    | 11 K  
4 | item_lin | Linear    | 11 K  


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.4515, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 1.451522946357727}